# Ordinary differential equation model
### Simon Frost (@sdwfrost)
### 2020-04-27
## Introduction

The classical ODE version of the SIR model is:

- Deterministic
- Continuous in time
- Continuous in state

## Libraries

In [ ]:
using DifferentialEquations
using SimpleDiffEq
using DataFrames
using StatsPlots
using BenchmarkTools

In [ ]:
readdir(); pwd()

## Transitions

The following function provides the derivatives of the model, which it changes in-place. State variables and parameters are unpacked from `u` and `p`; this incurs a slight performance hit, but makes the equations much easier to read.

In [ ]:
function sir_ode!(du,u,p,t)
    (S,I,R) = u
    (β,c,γ) = p
    N = S+I+R
    @inbounds begin
        du[1] = -β*c*I/N*S
        du[2] = β*c*I/N*S - γ*I
        du[3] = γ*I
    end
    nothing
end;

## Time domain

We set the timespan for simulations, `tspan`, initial conditions, `u0`, and parameter values, `p` (which are unpacked above as `[β,γ]`).

In [ ]:
δt = 0.1
tmax = 40.0
tspan = (0.0,tmax)
t = 0.0:δt:tmax;

## Initial conditions

In [ ]:
u0 = [990.0,10.0,0.0]

## Parameter values

In [ ]:
p = [0.05,10.0,0.25];

## Running the model

In [ ]:
prob_ode = ODEProblem(sir_ode!,u0,tspan,p)

In [ ]:
sol_ode = solve(prob_ode);

## Post-processing

We can convert the output to a dataframe for convenience.

In [ ]:
# sol_ode: 3 x sz
df_ode = DataFrame(sol_ode(t)')
df_ode[!,:t] = t;

## Plotting

We can now plot the results.

In [ ]:
@df df_ode plot(:t,
    [:x1 :x2 :x3],
    label=["S" "I" "R"],
    xlabel="Time",
    ylabel="Number")

## Benchmarking

In [ ]:
@benchmark solve(prob_ode)

In [ ]:
# Use 